In [ ]:
!pip install boto3 langchain langchain-pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.2/83.2 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 6.9 MB/s eta 0:00:00
  Attempting uninstall: aiohttp
    Found existing installation: aiohttp 3.11.10
    Uninstalling aiohttp-3.11.10:
      Successfully uninstalled aiohttp-3.11.10


In [ ]:
import boto3
import os


boto3_bedrock = boto3.client('bedrock-runtime',region_name='eu-central-1',aws_access_key_id='',aws_secret_access_key='')

kendra_client=boto3.client('kendra',region_name='eu-west-2',aws_access_key_id='',aws_secret_access_key='')


In [ ]:
from langchain.docstore.document import Document
from langchain.document_loaders import TextLoader
from langchain.embeddings import BedrockEmbeddings
from langchain_text_splitters import CharacterTextSplitter

In [ ]:
from langchain_community.retrievers import AmazonKendraRetriever

In [ ]:
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.1 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.12
    Uninstalling langchain-0.3.12:
      Successfully uninstalled langchain-0.3.12


In [ ]:
retriever = AmazonKendraRetriever(index_id="df30eeae-c085-4a56-abc9-7ca05f4db3ef",
                                  client =kendra_client,
                                  region_name="eu-west-2",
                                  credentials_profile_name="kendra",
                                  top_k=2,
                                  min_score_confidence=0.5
                                  )

response=retriever.get_relevant_documents("key features")
response

[Document(metadata={'result_id': '27249547-a33e-4756-ba57-3ae50bf1e2b7-688351d8-91ad-4125-a731-0c10df26f751', 'document_id': 's3://bucket-kendra-london/Product Description Flyer.docx', 'source': 'https://bucket-kendra-london.s3.eu-west-2.amazonaws.com/Product%20Description%20Flyer.docx', 'title': 'Product Description Flyer.docx', 'excerpt': 'Product Overview What is Our Recommendation Engine? Our Recommendation Engine uses advanced algorithms and AI to analyze customer behavior, preferences, and trends to provide personalized recommendations in real-time. Whether it’s suggesting products, services, or content, our engine ensures that each customer’s experience is uniquely tailored, leading to higher engagement and conversion rates. Key Features 1. Real-Time Personalization Deliver dynamic recommendations that adapt to user behavior instantly. 2. Intelligent Algorithms \xa0Utilize cutting-edge machine learning techniques to analyze user behavior and preferences. 3. Seamless Integration 

In [ ]:
from langchain import PromptTemplate

RAG_PROMPT_TEMPLATE = '''Here is some important context which can help inform the questions the Human asks.
Make sure to not make anything up to answer the question if it is not provided in the context.


{context}


Human: {human_input}

Assistant:
'''
PROMPT = PromptTemplate.from_template(RAG_PROMPT_TEMPLATE)

import json
human_input=input("Enter the question : ")
search_results =retriever.get_relevant_documents(human_input)
context_string = '\n\n'.join([f'Document {ind+1}: ' + i.page_content for ind, i in enumerate(search_results)])
prompt_data = PROMPT.format(human_input=human_input, context=context_string)
inputText=prompt_data
body_part=json.dumps({'inputText': inputText,
 'textGenerationConfig': {'maxTokenCount': 8192,
  'stopSequences': [],
  'temperature': 0,
  'topP': 1}})
response = boto3_bedrock.invoke_model(
    body=body_part,
    contentType="application/json",
    accept="application/json",
    modelId='amazon.titan-text-express-v1'
)
output_text=json.loads(response['body'].read())['results'][0]['outputText']
output_text

Enter the question : how can i contact with sales


'To contact with sales, please visit https://ecommend.tech or contact with sales@ecommend.tech.'